### Night Seven assignment - import HL7 message and convert to JSON 

Conceptually, this assigment requires the following:  
1. Read in the HL7 messages.
2. Create an object capable of accomplishing the following:
        a. Use the Segment identifiers as "Keys"     
        b. Use the Fields as "Values"
        c. Use Components with sub-components as "sub-Values"

In [1]:
import os
import pandas as pd
from pandas import Series, DataFrame
import numpy as np
import csv
import io
from UserDict import UserDict
import string
import simplejson
from numpy import genfromtxt

In [2]:
pd.set_option('display.mpl_style','default')


Below - usually a good idea to read a file (or files) before importing it (them), to make sure it (they) contain(s) data, that the data is not corrupted, etc.

In [3]:
msg = open('C:\Users\Bill\iPython\Night Seven\msg1.txt', 'r')
for file in msg:
    print '{', msg, '}'

{ <open file 'C:\\Users\\Bill\\iPython\\Night Seven\\msg1.txt', mode 'r' at 0x000000000B8C6930> }
{ <open file 'C:\\Users\\Bill\\iPython\\Night Seven\\msg1.txt', mode 'r' at 0x000000000B8C6930> }
{ <open file 'C:\\Users\\Bill\\iPython\\Night Seven\\msg1.txt', mode 'r' at 0x000000000B8C6930> }
{ <open file 'C:\\Users\\Bill\\iPython\\Night Seven\\msg1.txt', mode 'r' at 0x000000000B8C6930> }
{ <open file 'C:\\Users\\Bill\\iPython\\Night Seven\\msg1.txt', mode 'r' at 0x000000000B8C6930> }
{ <open file 'C:\\Users\\Bill\\iPython\\Night Seven\\msg1.txt', mode 'r' at 0x000000000B8C6930> }
{ <open file 'C:\\Users\\Bill\\iPython\\Night Seven\\msg1.txt', mode 'r' at 0x000000000B8C6930> }
{ <open file 'C:\\Users\\Bill\\iPython\\Night Seven\\msg1.txt', mode 'r' at 0x000000000B8C6930> }
{ <open file 'C:\\Users\\Bill\\iPython\\Night Seven\\msg1.txt', mode 'r' at 0x000000000B8C6930> }
{ <open file 'C:\\Users\\Bill\\iPython\\Night Seven\\msg1.txt', mode 'r' at 0x000000000B8C6930> }
{ <open file 'C:\\Us

In [4]:
hl7_msg = open('C:\Users\Bill\iPython\Night Seven\msg1.txt', 'r')
#for line in HL7_msg:
print '{', hl7_msg, '}'

{ <open file 'C:\\Users\\Bill\\iPython\\Night Seven\\msg1.txt', mode 'r' at 0x000000000B8C6A50> }


In [5]:
hl7_msg = open('C:\Users\Bill\iPython\Night Seven\msg1.txt', 'r')
for line in hl7_msg:
    #print line - if you don't include .strip(), there is a blank between each line
    print line.strip()

MSH|^~\&||ABCDE|||201507230021||ADT^A08|CAGTADM.1.10532994|P|2.1
EVN|A08|201507230021|||R.CA.NAH^HALL^NANCY^A^^^
PID|1||V000042610|M88604|BARTON^CLINTON^FRANCIS^^^||19510121|M|^^^^^|W|9 E CARTER ST APT A^^CARTERSVILLE^GA^30120^USA^^^BARTOW||(000)000-0000|(000)000-0000|ENG|D|OTH|V07016760770|000-00-0000
NK1|1|POSTEN^BOBBY^^^^|OT|102 S.H.E.I.L.D Floating Island^^CARTERSVILLE^GA^30120^USA^^^BARTOW|(000)000-0000|(000)000-0000
NK1|2|POSTEN^BOBBY^^^^|OT|102 S.H.E.I.L.D Floating Island^^CARTERSVILLE^GA^30120^USA^^^BARTOW|(000)000-0000|(000)000-0000
PV1|1|E|V.ER^^|EM|||WILSON^Wade^Deadpool^^^MD|.SELF^Referred^Self^^^^|MARKO^Cain^Bighelmet^^^MD|ER||||PR|WI|N||ER||03|||||||||||||||||||ABCDE^|back pain|REG|||201507222358
AL1|1|DA|F000004181^tramadol HCl^From ULTRAM|MO|JERKING|20140525
ACC|20150722^|11
GT1|1||BARTON^CLINTON^FRANCIS^^^||9 S.H.E.I.L.D Training Facility^^CARTERSVILLE^GA^30120^USA^^^BARTOW|(000)000-0000||19510121|M||SA|000-00-0000|||DISABLED|DISABLED^^DISABLED^GA^30092|(000)000-0000||

Below does not work - adds the { } to each line

In [6]:
hl7_msg = open('C:\Users\Bill\iPython\Night Seven\msg1.txt', 'r')
for line in hl7_msg:
    print '{', line.strip(), '}'

{ MSH|^~\&||ABCDE|||201507230021||ADT^A08|CAGTADM.1.10532994|P|2.1 }
{ EVN|A08|201507230021|||R.CA.NAH^HALL^NANCY^A^^^ }
{ PID|1||V000042610|M88604|BARTON^CLINTON^FRANCIS^^^||19510121|M|^^^^^|W|9 E CARTER ST APT A^^CARTERSVILLE^GA^30120^USA^^^BARTOW||(000)000-0000|(000)000-0000|ENG|D|OTH|V07016760770|000-00-0000 }
{ NK1|1|POSTEN^BOBBY^^^^|OT|102 S.H.E.I.L.D Floating Island^^CARTERSVILLE^GA^30120^USA^^^BARTOW|(000)000-0000|(000)000-0000 }
{ NK1|2|POSTEN^BOBBY^^^^|OT|102 S.H.E.I.L.D Floating Island^^CARTERSVILLE^GA^30120^USA^^^BARTOW|(000)000-0000|(000)000-0000 }
{ PV1|1|E|V.ER^^|EM|||WILSON^Wade^Deadpool^^^MD|.SELF^Referred^Self^^^^|MARKO^Cain^Bighelmet^^^MD|ER||||PR|WI|N||ER||03|||||||||||||||||||ABCDE^|back pain|REG|||201507222358 }
{ AL1|1|DA|F000004181^tramadol HCl^From ULTRAM|MO|JERKING|20140525 }
{ ACC|20150722^|11 }
{ GT1|1||BARTON^CLINTON^FRANCIS^^^||9 S.H.E.I.L.D Training Facility^^CARTERSVILLE^GA^30120^USA^^^BARTOW|(000)000-0000||19510121|M||SA|000-00-0000|||DISABLED|DISABLED^

In [7]:
with open('C:\Users\Bill\iPython\Night Seven\msg1.txt', 'rb') as csvfile:
    hl7_seg = csv.reader(csvfile, delimiter = '|')
    for row in hl7_seg:
        print row

['MSH', '^~\\&', '', 'ABCDE', '', '', '201507230021', '', 'ADT^A08', 'CAGTADM.1.10532994', 'P', '2.1']
['EVN', 'A08', '201507230021', '', '', 'R.CA.NAH^HALL^NANCY^A^^^']
['PID', '1', '', 'V000042610', 'M88604', 'BARTON^CLINTON^FRANCIS^^^', '', '19510121', 'M', '^^^^^', 'W', '9 E CARTER ST APT A^^CARTERSVILLE^GA^30120^USA^^^BARTOW', '', '(000)000-0000', '(000)000-0000', 'ENG', 'D', 'OTH', 'V07016760770', '000-00-0000']
['NK1', '1', 'POSTEN^BOBBY^^^^', 'OT', '102 S.H.E.I.L.D Floating Island^^CARTERSVILLE^GA^30120^USA^^^BARTOW', '(000)000-0000', '(000)000-0000']
['NK1', '2', 'POSTEN^BOBBY^^^^', 'OT', '102 S.H.E.I.L.D Floating Island^^CARTERSVILLE^GA^30120^USA^^^BARTOW', '(000)000-0000', '(000)000-0000']
['PV1', '1', 'E', 'V.ER^^', 'EM', '', '', 'WILSON^Wade^Deadpool^^^MD', '.SELF^Referred^Self^^^^', 'MARKO^Cain^Bighelmet^^^MD', 'ER', '', '', '', 'PR', 'WI', 'N', '', 'ER', '', '03', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', 'ABCDE^', 'back pain', 'REG', '', ''

Create the segments from the csv file.

In [8]:
csvfile = open('C:\Users\Bill\iPython\Night Seven\msg1.txt', 'rb')
# declare and create the datastructure
hl7_seg_lst = []
# read in each row
for row in csvfile:
    # strip the carriage return
    segment = row.strip()
    # append the datastructure
    hl7_seg_lst.append(segment)
# print the first three segments; confirm things worked    
hl7_seg_lst[0:3] 

['MSH|^~\\&||ABCDE|||201507230021||ADT^A08|CAGTADM.1.10532994|P|2.1',
 'EVN|A08|201507230021|||R.CA.NAH^HALL^NANCY^A^^^',
 'PID|1||V000042610|M88604|BARTON^CLINTON^FRANCIS^^^||19510121|M|^^^^^|W|9 E CARTER ST APT A^^CARTERSVILLE^GA^30120^USA^^^BARTOW||(000)000-0000|(000)000-0000|ENG|D|OTH|V07016760770|000-00-0000']

Create the fields from the segments.

In [9]:
hl7_fld_lst = []

for segment in hl7_seg_lst: 
    fields = segment.split('|')
    hl7_fld_lst.append(fields)
    
hl7_fld_lst[0:2]

[['MSH',
  '^~\\&',
  '',
  'ABCDE',
  '',
  '',
  '201507230021',
  '',
  'ADT^A08',
  'CAGTADM.1.10532994',
  'P',
  '2.1'],
 ['EVN', 'A08', '201507230021', '', '', 'R.CA.NAH^HALL^NANCY^A^^^']]

Create the components from the fields. Tried splitting only as mentioned in class on 10/5/15. Did not work.

In [10]:
hl7_fld_cmp_lst = []

for fields in hl7_fld_lst: 
    if '^' in fields:
        comp = fields.split('^')
        hl7_fld_cmp_lst.append(comp)
    else:
        hl7_fld_cmp_lst.append(fields)
        
hl7_fld_cmp_lst[0]

['MSH',
 '^~\\&',
 '',
 'ABCDE',
 '',
 '',
 '201507230021',
 '',
 'ADT^A08',
 'CAGTADM.1.10532994',
 'P',
 '2.1']

In [11]:
simplejson.dumps(hl7_fld_lst)

'[["MSH", "^~\\\\&", "", "ABCDE", "", "", "201507230021", "", "ADT^A08", "CAGTADM.1.10532994", "P", "2.1"], ["EVN", "A08", "201507230021", "", "", "R.CA.NAH^HALL^NANCY^A^^^"], ["PID", "1", "", "V000042610", "M88604", "BARTON^CLINTON^FRANCIS^^^", "", "19510121", "M", "^^^^^", "W", "9 E CARTER ST APT A^^CARTERSVILLE^GA^30120^USA^^^BARTOW", "", "(000)000-0000", "(000)000-0000", "ENG", "D", "OTH", "V07016760770", "000-00-0000"], ["NK1", "1", "POSTEN^BOBBY^^^^", "OT", "102 S.H.E.I.L.D Floating Island^^CARTERSVILLE^GA^30120^USA^^^BARTOW", "(000)000-0000", "(000)000-0000"], ["NK1", "2", "POSTEN^BOBBY^^^^", "OT", "102 S.H.E.I.L.D Floating Island^^CARTERSVILLE^GA^30120^USA^^^BARTOW", "(000)000-0000", "(000)000-0000"], ["PV1", "1", "E", "V.ER^^", "EM", "", "", "WILSON^Wade^Deadpool^^^MD", ".SELF^Referred^Self^^^^", "MARKO^Cain^Bighelmet^^^MD", "ER", "", "", "", "PR", "WI", "N", "", "ER", "", "03", "", "", "", "", "", "", "", "", "", "", "", "", "", "", "", "", "", "", "ABCDE^", "back pain", "REG

In [12]:
hl7_msg_2 = open('C:\Users\Bill\iPython\Night Seven\msg1.txt', 'r')
print '{' + '\n' + hl7_msg_2.read() + '}'

{
MSH|^~\&||ABCDE|||201507230021||ADT^A08|CAGTADM.1.10532994|P|2.1
EVN|A08|201507230021|||R.CA.NAH^HALL^NANCY^A^^^
PID|1||V000042610|M88604|BARTON^CLINTON^FRANCIS^^^||19510121|M|^^^^^|W|9 E CARTER ST APT A^^CARTERSVILLE^GA^30120^USA^^^BARTOW||(000)000-0000|(000)000-0000|ENG|D|OTH|V07016760770|000-00-0000
NK1|1|POSTEN^BOBBY^^^^|OT|102 S.H.E.I.L.D Floating Island^^CARTERSVILLE^GA^30120^USA^^^BARTOW|(000)000-0000|(000)000-0000
NK1|2|POSTEN^BOBBY^^^^|OT|102 S.H.E.I.L.D Floating Island^^CARTERSVILLE^GA^30120^USA^^^BARTOW|(000)000-0000|(000)000-0000
PV1|1|E|V.ER^^|EM|||WILSON^Wade^Deadpool^^^MD|.SELF^Referred^Self^^^^|MARKO^Cain^Bighelmet^^^MD|ER||||PR|WI|N||ER||03|||||||||||||||||||ABCDE^|back pain|REG|||201507222358
AL1|1|DA|F000004181^tramadol HCl^From ULTRAM|MO|JERKING|20140525
ACC|20150722^|11
GT1|1||BARTON^CLINTON^FRANCIS^^^||9 S.H.E.I.L.D Training Facility^^CARTERSVILLE^GA^30120^USA^^^BARTOW|(000)000-0000||19510121|M||SA|000-00-0000|||DISABLED|DISABLED^^DISABLED^GA^30092|(000)000-0000

In [13]:
with open('C:\Users\Bill\iPython\Night Seven\msg1.txt', 'r') as f:
    msg = f.readlines()
    print msg

['MSH|^~\\&||ABCDE|||201507230021||ADT^A08|CAGTADM.1.10532994|P|2.1\n', 'EVN|A08|201507230021|||R.CA.NAH^HALL^NANCY^A^^^\n', 'PID|1||V000042610|M88604|BARTON^CLINTON^FRANCIS^^^||19510121|M|^^^^^|W|9 E CARTER ST APT A^^CARTERSVILLE^GA^30120^USA^^^BARTOW||(000)000-0000|(000)000-0000|ENG|D|OTH|V07016760770|000-00-0000\n', 'NK1|1|POSTEN^BOBBY^^^^|OT|102 S.H.E.I.L.D Floating Island^^CARTERSVILLE^GA^30120^USA^^^BARTOW|(000)000-0000|(000)000-0000\n', 'NK1|2|POSTEN^BOBBY^^^^|OT|102 S.H.E.I.L.D Floating Island^^CARTERSVILLE^GA^30120^USA^^^BARTOW|(000)000-0000|(000)000-0000\n', 'PV1|1|E|V.ER^^|EM|||WILSON^Wade^Deadpool^^^MD|.SELF^Referred^Self^^^^|MARKO^Cain^Bighelmet^^^MD|ER||||PR|WI|N||ER||03|||||||||||||||||||ABCDE^|back pain|REG|||201507222358\n', 'AL1|1|DA|F000004181^tramadol HCl^From ULTRAM|MO|JERKING|20140525\n', 'ACC|20150722^|11\n', 'GT1|1||BARTON^CLINTON^FRANCIS^^^||9 S.H.E.I.L.D Training Facility^^CARTERSVILLE^GA^30120^USA^^^BARTOW|(000)000-0000||19510121|M||SA|000-00-0000|||DISABLED|

### Seemed to get heading in the right direction starting here.

Print the lines from the file with the End of Line '\n' removed.

In [14]:
with open('C:\Users\Bill\iPython\Night Seven\msg1.txt', 'r') as f:
    print '{'
    msg = f.readlines()

    for line in msg:
        # strip out the End of Line escape sequence
        line = line.rstrip('\n')
        print line
            
    print '}'

{
MSH|^~\&||ABCDE|||201507230021||ADT^A08|CAGTADM.1.10532994|P|2.1
EVN|A08|201507230021|||R.CA.NAH^HALL^NANCY^A^^^
PID|1||V000042610|M88604|BARTON^CLINTON^FRANCIS^^^||19510121|M|^^^^^|W|9 E CARTER ST APT A^^CARTERSVILLE^GA^30120^USA^^^BARTOW||(000)000-0000|(000)000-0000|ENG|D|OTH|V07016760770|000-00-0000
NK1|1|POSTEN^BOBBY^^^^|OT|102 S.H.E.I.L.D Floating Island^^CARTERSVILLE^GA^30120^USA^^^BARTOW|(000)000-0000|(000)000-0000
NK1|2|POSTEN^BOBBY^^^^|OT|102 S.H.E.I.L.D Floating Island^^CARTERSVILLE^GA^30120^USA^^^BARTOW|(000)000-0000|(000)000-0000
PV1|1|E|V.ER^^|EM|||WILSON^Wade^Deadpool^^^MD|.SELF^Referred^Self^^^^|MARKO^Cain^Bighelmet^^^MD|ER||||PR|WI|N||ER||03|||||||||||||||||||ABCDE^|back pain|REG|||201507222358
AL1|1|DA|F000004181^tramadol HCl^From ULTRAM|MO|JERKING|20140525
ACC|20150722^|11
GT1|1||BARTON^CLINTON^FRANCIS^^^||9 S.H.E.I.L.D Training Facility^^CARTERSVILLE^GA^30120^USA^^^BARTOW|(000)000-0000||19510121|M||SA|000-00-0000|||DISABLED|DISABLED^^DISABLED^GA^30092|(000)000-0000

First split of the lines - on the '|'

In [15]:
with open('C:\Users\Bill\iPython\Night Seven\msg1.txt', 'r') as f:
    print '{'
    msg = f.readlines()

    for line in msg:
        line = line.rstrip('\n')
        # split on the '|'
        segm = line.split('|')
        print segm
            
    print '}' 

{
['MSH', '^~\\&', '', 'ABCDE', '', '', '201507230021', '', 'ADT^A08', 'CAGTADM.1.10532994', 'P', '2.1']
['EVN', 'A08', '201507230021', '', '', 'R.CA.NAH^HALL^NANCY^A^^^']
['PID', '1', '', 'V000042610', 'M88604', 'BARTON^CLINTON^FRANCIS^^^', '', '19510121', 'M', '^^^^^', 'W', '9 E CARTER ST APT A^^CARTERSVILLE^GA^30120^USA^^^BARTOW', '', '(000)000-0000', '(000)000-0000', 'ENG', 'D', 'OTH', 'V07016760770', '000-00-0000']
['NK1', '1', 'POSTEN^BOBBY^^^^', 'OT', '102 S.H.E.I.L.D Floating Island^^CARTERSVILLE^GA^30120^USA^^^BARTOW', '(000)000-0000', '(000)000-0000']
['NK1', '2', 'POSTEN^BOBBY^^^^', 'OT', '102 S.H.E.I.L.D Floating Island^^CARTERSVILLE^GA^30120^USA^^^BARTOW', '(000)000-0000', '(000)000-0000']
['PV1', '1', 'E', 'V.ER^^', 'EM', '', '', 'WILSON^Wade^Deadpool^^^MD', '.SELF^Referred^Self^^^^', 'MARKO^Cain^Bighelmet^^^MD', 'ER', '', '', '', 'PR', 'WI', 'N', '', 'ER', '', '03', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', 'ABCDE^', 'back pain', 'REG', '', 

Tried to create a 'key' - 'value' pair...

In [16]:
with open('C:\Users\Bill\iPython\Night Seven\msg1.txt', 'r') as f:
    print '{'
    msg = f.readlines()

    lc = 0
    for line in msg:
        lc = lc + 1
        line = 'SEG' + str(lc) + ':|' + line
        line = line.rstrip('\n')
        # split on the '|'
        segm = line.split('|')
        #segm = 'Seg' + str(lc) + segm
        print segm
            
    print '}' 
    print lc

{
['SEG1:', 'MSH', '^~\\&', '', 'ABCDE', '', '', '201507230021', '', 'ADT^A08', 'CAGTADM.1.10532994', 'P', '2.1']
['SEG2:', 'EVN', 'A08', '201507230021', '', '', 'R.CA.NAH^HALL^NANCY^A^^^']
['SEG3:', 'PID', '1', '', 'V000042610', 'M88604', 'BARTON^CLINTON^FRANCIS^^^', '', '19510121', 'M', '^^^^^', 'W', '9 E CARTER ST APT A^^CARTERSVILLE^GA^30120^USA^^^BARTOW', '', '(000)000-0000', '(000)000-0000', 'ENG', 'D', 'OTH', 'V07016760770', '000-00-0000']
['SEG4:', 'NK1', '1', 'POSTEN^BOBBY^^^^', 'OT', '102 S.H.E.I.L.D Floating Island^^CARTERSVILLE^GA^30120^USA^^^BARTOW', '(000)000-0000', '(000)000-0000']
['SEG5:', 'NK1', '2', 'POSTEN^BOBBY^^^^', 'OT', '102 S.H.E.I.L.D Floating Island^^CARTERSVILLE^GA^30120^USA^^^BARTOW', '(000)000-0000', '(000)000-0000']
['SEG6:', 'PV1', '1', 'E', 'V.ER^^', 'EM', '', '', 'WILSON^Wade^Deadpool^^^MD', '.SELF^Referred^Self^^^^', 'MARKO^Cain^Bighelmet^^^MD', 'ER', '', '', '', 'PR', 'WI', 'N', '', 'ER', '', '03', '', '', '', '', '', '', '', '', '', '', '', '', '', 

Second split of the lines - on the '^'

In [17]:
with open('C:\Users\Bill\iPython\Night Seven\msg1.txt', 'r') as f:
    
    print '{'
    msg = f.readlines()

    for line in msg:
        line = line.rstrip('\n')
        segm = line.split('|')
        
        for line in segm:
            fld = line.split('^')
            print fld
            
    print '}'        
        

{
['MSH']
['', '~\\&']
['']
['ABCDE']
['']
['']
['201507230021']
['']
['ADT', 'A08']
['CAGTADM.1.10532994']
['P']
['2.1']
['EVN']
['A08']
['201507230021']
['']
['']
['R.CA.NAH', 'HALL', 'NANCY', 'A', '', '', '']
['PID']
['1']
['']
['V000042610']
['M88604']
['BARTON', 'CLINTON', 'FRANCIS', '', '', '']
['']
['19510121']
['M']
['', '', '', '', '', '']
['W']
['9 E CARTER ST APT A', '', 'CARTERSVILLE', 'GA', '30120', 'USA', '', '', 'BARTOW']
['']
['(000)000-0000']
['(000)000-0000']
['ENG']
['D']
['OTH']
['V07016760770']
['000-00-0000']
['NK1']
['1']
['POSTEN', 'BOBBY', '', '', '', '']
['OT']
['102 S.H.E.I.L.D Floating Island', '', 'CARTERSVILLE', 'GA', '30120', 'USA', '', '', 'BARTOW']
['(000)000-0000']
['(000)000-0000']
['NK1']
['2']
['POSTEN', 'BOBBY', '', '', '', '']
['OT']
['102 S.H.E.I.L.D Floating Island', '', 'CARTERSVILLE', 'GA', '30120', 'USA', '', '', 'BARTOW']
['(000)000-0000']
['(000)000-0000']
['PV1']
['1']
['E']
['V.ER', '', '']
['EM']
['']
['']
['WILSON', 'Wade', 'Deadpool', 

Third split of the lines - on the ','

In [18]:
with open('C:\Users\Bill\iPython\Night Seven\msg1.txt', 'r') as f:
    
    print '{'
    msg = f.readlines()

    for line in msg:
        line = line.rstrip('\n')
        segm = line.split('|')
        
        for line in segm:
            fld = line.split('^')
            
            for line in fld:
                comp = line.split(',')
                print comp
    print '}'        
        

{
['MSH']
['']
['~\\&']
['']
['ABCDE']
['']
['']
['201507230021']
['']
['ADT']
['A08']
['CAGTADM.1.10532994']
['P']
['2.1']
['EVN']
['A08']
['201507230021']
['']
['']
['R.CA.NAH']
['HALL']
['NANCY']
['A']
['']
['']
['']
['PID']
['1']
['']
['V000042610']
['M88604']
['BARTON']
['CLINTON']
['FRANCIS']
['']
['']
['']
['']
['19510121']
['M']
['']
['']
['']
['']
['']
['']
['W']
['9 E CARTER ST APT A']
['']
['CARTERSVILLE']
['GA']
['30120']
['USA']
['']
['']
['BARTOW']
['']
['(000)000-0000']
['(000)000-0000']
['ENG']
['D']
['OTH']
['V07016760770']
['000-00-0000']
['NK1']
['1']
['POSTEN']
['BOBBY']
['']
['']
['']
['']
['OT']
['102 S.H.E.I.L.D Floating Island']
['']
['CARTERSVILLE']
['GA']
['30120']
['USA']
['']
['']
['BARTOW']
['(000)000-0000']
['(000)000-0000']
['NK1']
['2']
['POSTEN']
['BOBBY']
['']
['']
['']
['']
['OT']
['102 S.H.E.I.L.D Floating Island']
['']
['CARTERSVILLE']
['GA']
['30120']
['USA']
['']
['']
['BARTOW']
['(000)000-0000']
['(000)000-0000']
['PV1']
['1']
['E']
['V.ER']
['']

Omitting any completely empty fields and components from printing.

In [19]:
with open('C:\Users\Bill\iPython\Night Seven\msg1.txt', 'r') as f:
    
    print '{'
    msg = f.readlines()

    for line in msg:
        line = line.rstrip('\n')
        segm = line.split('|')
        
        for line in segm:
            fld = line.split('^')
            
            for line in fld:
                comp = line.split(',')
                if comp != ['']:
                    print comp
    print '}'   

{
['MSH']
['~\\&']
['ABCDE']
['201507230021']
['ADT']
['A08']
['CAGTADM.1.10532994']
['P']
['2.1']
['EVN']
['A08']
['201507230021']
['R.CA.NAH']
['HALL']
['NANCY']
['A']
['PID']
['1']
['V000042610']
['M88604']
['BARTON']
['CLINTON']
['FRANCIS']
['19510121']
['M']
['W']
['9 E CARTER ST APT A']
['CARTERSVILLE']
['GA']
['30120']
['USA']
['BARTOW']
['(000)000-0000']
['(000)000-0000']
['ENG']
['D']
['OTH']
['V07016760770']
['000-00-0000']
['NK1']
['1']
['POSTEN']
['BOBBY']
['OT']
['102 S.H.E.I.L.D Floating Island']
['CARTERSVILLE']
['GA']
['30120']
['USA']
['BARTOW']
['(000)000-0000']
['(000)000-0000']
['NK1']
['2']
['POSTEN']
['BOBBY']
['OT']
['102 S.H.E.I.L.D Floating Island']
['CARTERSVILLE']
['GA']
['30120']
['USA']
['BARTOW']
['(000)000-0000']
['(000)000-0000']
['PV1']
['1']
['E']
['V.ER']
['EM']
['WILSON']
['Wade']
['Deadpool']
['MD']
['.SELF']
['Referred']
['Self']
['MARKO']
['Cain']
['Bighelmet']
['MD']
['ER']
['PR']
['WI']
['N']
['ER']
['03']
['ABCDE']
['back pain']
['REG']
['20150

Added loop counters - trying to think about having the numbering we need for json

In [20]:
with open('C:\Users\Bill\iPython\Night Seven\msg1.txt', 'r') as f:
    
    print '{'
    msg = f.readlines()
    
    lc = 0
    fc = 0
    cc = 0
    for line in msg:
        lc = lc + 1
        line = line.rstrip('\n')
        segm = line.split('|')
        
        for line in segm:
            fld = line.split('^')
            fc = fc + 1
           
            for line in fld:
                comp = line.split(',')
                cc = cc + 1
                # will NOT print any field or component with '' for value.
                if comp != ['']:
                    print comp
    print '}'
    print 'Line Count: ' + str(lc)
    print 'Field Count: ' + str(fc)
    print 'Component Count: ' + str(cc)

{
['MSH']
['~\\&']
['ABCDE']
['201507230021']
['ADT']
['A08']
['CAGTADM.1.10532994']
['P']
['2.1']
['EVN']
['A08']
['201507230021']
['R.CA.NAH']
['HALL']
['NANCY']
['A']
['PID']
['1']
['V000042610']
['M88604']
['BARTON']
['CLINTON']
['FRANCIS']
['19510121']
['M']
['W']
['9 E CARTER ST APT A']
['CARTERSVILLE']
['GA']
['30120']
['USA']
['BARTOW']
['(000)000-0000']
['(000)000-0000']
['ENG']
['D']
['OTH']
['V07016760770']
['000-00-0000']
['NK1']
['1']
['POSTEN']
['BOBBY']
['OT']
['102 S.H.E.I.L.D Floating Island']
['CARTERSVILLE']
['GA']
['30120']
['USA']
['BARTOW']
['(000)000-0000']
['(000)000-0000']
['NK1']
['2']
['POSTEN']
['BOBBY']
['OT']
['102 S.H.E.I.L.D Floating Island']
['CARTERSVILLE']
['GA']
['30120']
['USA']
['BARTOW']
['(000)000-0000']
['(000)000-0000']
['PV1']
['1']
['E']
['V.ER']
['EM']
['WILSON']
['Wade']
['Deadpool']
['MD']
['.SELF']
['Referred']
['Self']
['MARKO']
['Cain']
['Bighelmet']
['MD']
['ER']
['PR']
['WI']
['N']
['ER']
['03']
['ABCDE']
['back pain']
['REG']
['20150

In [21]:
with open('C:\Users\Bill\iPython\Night Seven\msg1.txt', 'r') as f:
    
    print '{'
    # msg = f.readlines()
    # changed to 'for line in f:' after reading about how it loads all the lines into a list before reading starts
    
    lc = 0
    fc = 0
    cc = 0
    for line in f:
        lc = lc + 1
        line = line.rstrip('\n')
        segm = line.split('|')
        
        for line in segm:
            fld = line.split('^')
            fc = fc + 1
           
            for line in fld:
                comp = line.split(',')
                cc = cc + 1
                if comp != ['']:
                    print comp
    print '}'
    print 'Line Count: ' + str(lc)
    print 'Field Count: ' + str(fc)
    print 'Component Count: ' + str(cc)

{
['MSH']
['~\\&']
['ABCDE']
['201507230021']
['ADT']
['A08']
['CAGTADM.1.10532994']
['P']
['2.1']
['EVN']
['A08']
['201507230021']
['R.CA.NAH']
['HALL']
['NANCY']
['A']
['PID']
['1']
['V000042610']
['M88604']
['BARTON']
['CLINTON']
['FRANCIS']
['19510121']
['M']
['W']
['9 E CARTER ST APT A']
['CARTERSVILLE']
['GA']
['30120']
['USA']
['BARTOW']
['(000)000-0000']
['(000)000-0000']
['ENG']
['D']
['OTH']
['V07016760770']
['000-00-0000']
['NK1']
['1']
['POSTEN']
['BOBBY']
['OT']
['102 S.H.E.I.L.D Floating Island']
['CARTERSVILLE']
['GA']
['30120']
['USA']
['BARTOW']
['(000)000-0000']
['(000)000-0000']
['NK1']
['2']
['POSTEN']
['BOBBY']
['OT']
['102 S.H.E.I.L.D Floating Island']
['CARTERSVILLE']
['GA']
['30120']
['USA']
['BARTOW']
['(000)000-0000']
['(000)000-0000']
['PV1']
['1']
['E']
['V.ER']
['EM']
['WILSON']
['Wade']
['Deadpool']
['MD']
['.SELF']
['Referred']
['Self']
['MARKO']
['Cain']
['Bighelmet']
['MD']
['ER']
['PR']
['WI']
['N']
['ER']
['03']
['ABCDE']
['back pain']
['REG']
['20150

In [23]:
with open('C:\Users\Bill\iPython\Night Seven\msg1.txt', 'r') as f:
    
    print '{'
    
    lc = 0
    fc = 0
    cc = 0
    for line in f:
        lc = lc + 1
        line = line.rstrip('\n')
        segm = line.split('|')
        
        for line in segm:
            fld = line.split('^')
            fc = fc + 1
        
            for line in fld:
                comp = line.split(',')
                cc = cc + 1
                # print everything...comment out the next line
                # if comp != ['']:
           
                print comp
    
    print '}'
    print 'Line Count: ' + str(lc)
    print 'Field Count: ' + str(fc)
    print 'Component Count: ' + str(cc)

{
['MSH']
['']
['~\\&']
['']
['ABCDE']
['']
['']
['201507230021']
['']
['ADT']
['A08']
['CAGTADM.1.10532994']
['P']
['2.1']
['EVN']
['A08']
['201507230021']
['']
['']
['R.CA.NAH']
['HALL']
['NANCY']
['A']
['']
['']
['']
['PID']
['1']
['']
['V000042610']
['M88604']
['BARTON']
['CLINTON']
['FRANCIS']
['']
['']
['']
['']
['19510121']
['M']
['']
['']
['']
['']
['']
['']
['W']
['9 E CARTER ST APT A']
['']
['CARTERSVILLE']
['GA']
['30120']
['USA']
['']
['']
['BARTOW']
['']
['(000)000-0000']
['(000)000-0000']
['ENG']
['D']
['OTH']
['V07016760770']
['000-00-0000']
['NK1']
['1']
['POSTEN']
['BOBBY']
['']
['']
['']
['']
['OT']
['102 S.H.E.I.L.D Floating Island']
['']
['CARTERSVILLE']
['GA']
['30120']
['USA']
['']
['']
['BARTOW']
['(000)000-0000']
['(000)000-0000']
['NK1']
['2']
['POSTEN']
['BOBBY']
['']
['']
['']
['']
['OT']
['102 S.H.E.I.L.D Floating Island']
['']
['CARTERSVILLE']
['GA']
['30120']
['USA']
['']
['']
['BARTOW']
['(000)000-0000']
['(000)000-0000']
['PV1']
['1']
['E']
['V.ER']
['']

#### Failed stuff

In [24]:
with open('C:\Users\Bill\iPython\Night Seven\msg1.txt', 'r') as f:
    
    print '{'
    msg = f.readlines()

    fc = 0
    for line in msg:
        line = line.rstrip('\n')
        segm = line.split('|')
        
        for line in segm:
            fld = line.split('^')
            fc = fc + 1
            fld = 'FLD' + str(fc) + ': ' + fld
            
            for line in fld:
                comp = line.split(',')
                if comp != ['']:
                    print comp
    print '}'   

{


TypeError: cannot concatenate 'str' and 'list' objects

In [25]:
with open('C:\Users\Bill\iPython\Night Seven\msg1.txt', 'r') as f:
    msg = [f.readlines()]
    print '{'
    print msg
    print '}'

{
[['MSH|^~\\&||ABCDE|||201507230021||ADT^A08|CAGTADM.1.10532994|P|2.1\n', 'EVN|A08|201507230021|||R.CA.NAH^HALL^NANCY^A^^^\n', 'PID|1||V000042610|M88604|BARTON^CLINTON^FRANCIS^^^||19510121|M|^^^^^|W|9 E CARTER ST APT A^^CARTERSVILLE^GA^30120^USA^^^BARTOW||(000)000-0000|(000)000-0000|ENG|D|OTH|V07016760770|000-00-0000\n', 'NK1|1|POSTEN^BOBBY^^^^|OT|102 S.H.E.I.L.D Floating Island^^CARTERSVILLE^GA^30120^USA^^^BARTOW|(000)000-0000|(000)000-0000\n', 'NK1|2|POSTEN^BOBBY^^^^|OT|102 S.H.E.I.L.D Floating Island^^CARTERSVILLE^GA^30120^USA^^^BARTOW|(000)000-0000|(000)000-0000\n', 'PV1|1|E|V.ER^^|EM|||WILSON^Wade^Deadpool^^^MD|.SELF^Referred^Self^^^^|MARKO^Cain^Bighelmet^^^MD|ER||||PR|WI|N||ER||03|||||||||||||||||||ABCDE^|back pain|REG|||201507222358\n', 'AL1|1|DA|F000004181^tramadol HCl^From ULTRAM|MO|JERKING|20140525\n', 'ACC|20150722^|11\n', 'GT1|1||BARTON^CLINTON^FRANCIS^^^||9 S.H.E.I.L.D Training Facility^^CARTERSVILLE^GA^30120^USA^^^BARTOW|(000)000-0000||19510121|M||SA|000-00-0000|||DISABL

In [26]:
simplejson.dump(msg, separators=('[', ']'))

TypeError: dump() takes at least 2 arguments (2 given)